In [1]:
import pyodbc
import pandas as pd
from IPython.display import display, Markdown

In [2]:
server = 'covid.ebmdatalab.net,1433'
database = 'OPENCoronaExport' 
username = 'SA'
password = 'ahsjdkaJAMSHDA123[' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [54]:
qof_clusters = pd.read_csv('../data/QoFClusteres_CTV3Codes - Sheet1.csv')
chd_codes = qof_clusters.loc[qof_clusters['ClusterId']=='CHD','CTV3Code']
chd_codes.head()

3562    14A3.
3563    14A4.
3564    G300.
3565    G301.
3566    G3010
Name: CTV3Code, dtype: object

In [56]:
def codes_to_sql_where(col_name, code_list):
    where = ""
    i = 0
    for code in code_list:
        if i == 0:
            where = where + f"{col_name} = '{code}'"
        else:
            where = where + f" OR {col_name} = '{code}'"
        i+=1
    return where

In [57]:
codes_to_sql_where("CTV3Code", chd_codes.head())

"CTV3Code = '14A3.' OR CTV3Code = '14A4.' OR CTV3Code = 'G300.' OR CTV3Code = 'G301.' OR CTV3Code = 'G3010'"

In [61]:
codes_where = codes_to_sql_where("CTV3Code", chd_codes)
query = f'''
SELECT Patient_ID
FROM CodedEvent
WHERE {codes_where}
'''
df = pd.read_sql(query, cnxn, index_col='Patient_ID')
df['chd'] = True
df

,chd
Patient_ID,
201,True
201,True
201,True
201,True
201,True
...,...
1434503,True
1434503,True
1434504,True
